In [14]:
import os
import tifffile
import numpy as np
import zarr

from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

from imjoy import api
from imjoy_rpc import register_default_codecs

def read_tiff_files(folder_path):
    tiff_files = [f for f in os.listdir(folder_path) if f.endswith('.tif') or f.endswith('.tiff')]
    tiff_files.sort()

    tiff_data_list = []
    for file_name in tiff_files:
        file_path = os.path.join(folder_path, file_name)
        tiff_data = tifffile.imread(file_path)
        tiff_data_list.append(tiff_data)

    tiff_array = np.stack(tiff_data_list)

    return tiff_array

# Example usage
folder_path = '/local/data/alberto/TXM/P-1C_267_rec/'
tiff_array = read_tiff_files(folder_path)


In [ ]:
output_path = '/local/data/alberto/TXM/test.zarr'

store = parse_url(output_path, mode="w").store
root = zarr.group(store=store)
print(help(write_image))

write_image(image=tiff_array, group=root, axes="zyx", storage_options=dict(chunks=(1,128, 128)))

In [12]:
register_default_codecs()

class Plugin:
    def __init__(self, images, view_state=None):
        if not isinstance(images, list):
            images = [images]
        self.images = images
        self.view_state = view_state

    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(
            type="vizarr", src="https://hms-dbmi.github.io/vizarr"
        )
        if self.view_state:
            await viewer.set_view_state(self.view_state)
        for img in self.images:
            await viewer.add_image(img)


def run_vizarr(images, view_state=None):
    api.export(Plugin(images, view_state))

In [13]:
#z_grp = zarr.open(store, mode="r")
data = zarr.open('/local/data/alberto/TXM/test.zarr', mode='r')#, dtype='float')
img = { "source": data}

run_vizarr(img)

<IPython.core.display.Javascript object>

here
